<a href="https://colab.research.google.com/github/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/blob/main/Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchinfo --quiet

!pip install wandb --quiet

import os
os.environ["WANDB_API_KEY"] = "571001027d1947e02fe01237fe2d0bf5bbf8071b"
os.environ["WANDB_NOTEBOOK_NAME"] = "Experiments"

     |████████████████████████████████| 1.9 MB 8.2 MB/s 
     |████████████████████████████████| 168 kB 15.0 MB/s 
     |████████████████████████████████| 182 kB 14.8 MB/s 
     |████████████████████████████████| 62 kB 441 kB/s 
     |████████████████████████████████| 168 kB 45.0 MB/s 
     |████████████████████████████████| 166 kB 7.1 MB/s 
     |████████████████████████████████| 166 kB 11.3 MB/s 
     |████████████████████████████████| 162 kB 43.4 MB/s 
     |████████████████████████████████| 162 kB 31.4 MB/s 
     |████████████████████████████████| 158 kB 15.1 MB/s 
     |████████████████████████████████| 157 kB 25.0 MB/s 
     |████████████████████████████████| 157 kB 61.0 MB/s 
     |████████████████████████████████| 157 kB 57.1 MB/s 
     |████████████████████████████████| 157 kB 32.0 MB/s 
     |████████████████████████████████| 157 kB 31.5 MB/s 
     |████████████████████████████████| 157 kB 62.5 MB/s 
     |████████████████████████████████| 157 kB 49.6 MB/s 
     |████████████

In [2]:
import zipfile
import numpy as np


!pip install torchinfo --quiet

!git clone https://github.com/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/

%cd Time-Series-Anomaly-Detection-An-experimental-survey

%mkdir data

# %mkdir ./data/SMD
# !python USAD/gdrivedl.py https://drive.google.com/file/d/18JNYBsaX7tu0Qfgo92nCBCklv471L1xB data/SMD
# with zipfile.ZipFile("./data/SMD/SMD.zip", 'r') as zip_ref:
#     zip_ref.extractall("./data/")

# %mkdir ./data/MSL
# !python USAD/gdrivedl.py https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3 data/MSL
# with zipfile.ZipFile("./data/MSL/MSL.zip", 'r') as zip_ref:
#     zip_ref.extractall("./data/")

!mkdir data/SWAT
#normal period
!python USAD/gdrivedl.py https://drive.google.com/open?id=1rVJ5ry5GG-ZZi5yI4x9lICB8VhErXwCw data/SWAT
#anomalies
!python USAD/gdrivedl.py https://drive.google.com/open?id=1iDYc0OEmidN712fquOBRFjln90SbpaE7 data/SWAT

Cloning into 'Time-Series-Anomaly-Detection-An-experimental-survey'...
remote: Enumerating objects: 1099, done.
remote: Counting objects: 100% (582/582), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 1099 (delta 396), reused 533 (delta 366), pack-reused 517
Receiving objects: 100% (1099/1099), 3.04 MiB | 16.12 MiB/s, done.
Resolving deltas: 100% (669/669), done.
/content/Time-Series-Anomaly-Detection-An-experimental-survey
data/SWAT/SWaT_Dataset_Normal_v1.csv
[==================================================] 163.77MB/163.77MB
data/SWAT/SWaT_Dataset_Attack_v0.csv
[==================================================] 127.27MB/127.27MB


In [3]:
### NEW USAD 
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'SWAT', #SWAT, MSL, SMD, NAB
    'DATAPATH': None, #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
    'SEQ_LEN': 100,
    'STEP': 100,
    'HIDDEN_SIZE': 100, #Needed for USAD method
    'LR': 0.001,
    'EPOCHS': 1, 
    'VERBOSE': True,
    'LOGGER': True
}
method = ADMethod(name = 'USAD', config = configuration)
train_history = method.train()
_, score = method.test()
for th in np.linspace(0,1,10,endpoint=False):
  rep = method.results(threshold = th, plot = False)

method.close_run()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find Experiments.


Initializing...


wandb: Currently logged in as: michiamoantonio. Use `wandb login --relogin` to force relogin


/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:25: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.load_data()
/content/Time-Series-Anomaly-Detection-An-experimental-survey/datafactory.py:25: DtypeWarning: Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.
  self.load_data()
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Data preprocessing and method configuration finished in 31.546733856201172 sec.
Model summary: 
Layer (type:depth-idx)                   Param #
UsadModel                                --
├─UsadEncoder: 1-1                       --
│    └─Linear: 2-1                       13,007,550
│    └─Linear: 2-2                       3,252,525
│    └─Linear: 2-3                       12,760,000
│    └─ReLU: 2-4                         --
├─UsadDecoder: 1-2                       --
│    └─Linear: 2-5                       12,751,275
│    └─Linear: 2-6                       3,253,800
│    └─Linear: 2-7                       13,010,100
│    └─ReLU: 2-8                         --
│    └─Sigmoid: 2-9                      --
├─UsadDecoder: 1-3                       --
│    └─Linear: 2-10                      12,751,275
│    └─Linear: 2-11                      3,253,800
│    └─Linear: 2-12                      13,010,100
│    └─ReLU: 2-13                        --
│    └─Sigmoid: 2-14                  

...

              precision    recall  f1-score   support

       False       0.95      1.00      0.97      3917
        True       1.00      0.62      0.76       582

    accuracy                           0.95      4499
   macro avg       0.97      0.81      0.87      4499
weighted avg       0.95      0.95      0.95      4499

Computing results... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



              precision    recall  f1-score   support

       False       0.95      1.00      0.97      3917
        True       1.00      0.62      0.76       582

    accuracy                           0.95      4499
   macro avg       0.97      0.81      0.87      4499
weighted avg       0.95      0.95      0.95      4499

Computing results... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DtypeWarning:

Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.



              precision    recall  f1-score   support

       False       0.95      1.00      0.97      3917
        True       1.00      0.62      0.76       582

    accuracy                           0.95      4499
   macro avg       0.97      0.81      0.87      4499
weighted avg       0.95      0.95      0.95      4499



Run closed.
Best accuracy is 0.9551011335852412 with threshold 0.4
Best Avg f1-score is 0.8826566449436368 with threshold 0.4
Best True f1-score is 0.7904564315352697 with threshold 0.4


F1_Avg,▁▅▆▇██████
F1_True,▁▃▅▆██████
Precision_Avg,▁▂▃▄██████
Precision_True,▁▂▃▅██████
Recall_Avg,▁▆▇█▇▇▇▇▇▇
Recall_True,█▇▆▄▂▁▁▁▁▁
accuracy,▁▅▇▇██████
epoch,▁
loss1,▁
loss2,▁
loss_sum,▁


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.metrics import classification_report
import plotly.express as px
import plotly.graph_objects as go
settings = configuration
def results(threshold: float, plot: bool, scores):
    print("=====================================================================")
    print("Computing results... ") 
		
		
    window_size = settings['SEQ_LEN']
    step = settings['STEP']
    
    attack = pd.read_csv("./data/SWAT/SWaT_Dataset_Attack_v0.csv", sep=";")
    ground_truth = attack.values[:,-1]
    ground_truth = np.array([False if el == "Normal" else True for el in ground_truth])
    limit = int(len(ground_truth)*0.6)
    ground_windows = ground_truth[np.arange(window_size)[None, :] + np.arange(0,limit-window_size, step)[:, None]]
    ground = np.array([True if el.sum() > 0 else False for el in ground_windows])
    
    scaler = MinMaxScaler()
    s = scaler.fit_transform(np.array(scores).reshape(-1, 1))
    anomalies = np.array([True if el > threshold else False for el in s])

    gt = ground
    pred = anomalies.copy()
    anomaly_state = False
    for i in range(len(gt)):
      if gt[i] == 1 and pred[i] == 1 and not anomaly_state:
        anomaly_state = True
        for j in range(i, 0, -1):
          if gt[j] == 0:
            break
          else:
            if pred[j] == 0:
                pred[j] = 1
        for j in range(i, len(gt)):
            if gt[j] == 0:
                break
            else:
                if pred[j] == 0:
                    pred[j] = 1
      elif gt[i] == 0:
        anomaly_state = False
      if anomaly_state:
        pred[i] = 1



    report = classification_report(ground, anomalies) #, output_dict=True)
    rep2 = classification_report(ground, pred)
    print(report)
    print(rep2)
    # print(classification_report(ground, anomalies, output_dict=False))
    
    # s_scaled = s.reshape(-1)
    # tp = np.logical_and(np.array(s_scaled) >= threshold,  ground == True)
    # fn = np.logical_and(np.array(s_scaled) < threshold,  ground == True)
    # fp = np.logical_and(np.array(s_scaled) >= threshold,  ground == False)
    
    # fig = go.Figure()
    # fig.add_hline(y=threshold, line_color="#aaaaaa", name="threshold", line_dash="dash")
    # fig.add_trace(go.Scatter(x=np.arange(len(s)), y=s_scaled, mode='lines', name='Anomaly score', line_color="#515ad6"))
    # fig.add_trace(go.Scatter(x=np.arange(len(s))[tp], y=ground[tp]*s_scaled[tp], mode='markers', name='True positives', line_color="green"))
    # fig.add_trace(go.Scatter(x=np.arange(len(s))[fn], y=ground[fn]*threshold, mode='markers', name='False negatives', line_color="red"))
    # fig.add_trace(go.Scatter(x=np.arange(len(s))[fp], y=s_scaled[fp], mode='markers', name='False positives', line_color="#000000", marker=dict(size=10, symbol=34, line=dict(width=2))))
    # fig.update_layout(plot_bgcolor="white")
    # fig.update_xaxes(showgrid=False, gridwidth=1, gridcolor='LightGrey', showline=True, linewidth=2, linecolor='DarkGrey')
    # fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', showline=True, linewidth=2, linecolor='DarkGrey')
    # fig.show()
    
    


In [ ]:
### NEW DEEPANT 
from AnomalyDetectionMethodClass import ADMethod

configuration = {
    'DATASET': 'MSL', #SWAT, MSL, SMD, NAB
    'DATAPATH': None, #"realKnownCause/machine_temperature_system_failure.csv", #Only needed with NAB
    'SEQ_LEN': 100,
    'STEP': 100,
    'HIDDEN_SIZE': None, #Needed for USAD method
    'LR': 0.00001,
    'EPOCHS': 80, #50 for SWAT
    'VERBOSE': True,
    'CENTILE': 1
}
method = ADMethod(name = 'DEEPANT', config = configuration)
train_losses = method.train()
predictions, score = method.test()
rep = method.results(0.6, True)

Initializing...
Data preprocessing and method configuration finished in 0.14605140686035156 sec.
Model summary: 
Layer (type:depth-idx)                   Param #
DeepAnt                                  --
├─Sequential: 1-1                        --
│    └─Conv1d: 2-1                       5,312
│    └─ReLU: 2-2                         --
│    └─MaxPool1d: 2-3                    --
├─Sequential: 1-2                        --
│    └─Conv1d: 2-4                       3,104
│    └─ReLU: 2-5                         --
│    └─MaxPool1d: 2-6                    --
├─Flatten: 1-3                           --
├─Sequential: 1-4                        --
│    └─Linear: 2-7                       29,480
│    └─ReLU: 2-8                         --
│    └─Dropout: 2-9                      --
├─Linear: 1-5                            2,255
Total params: 40,151
Trainable params: 40,151
Non-trainable params: 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1/80: train_loss:0.08260278403759003
Epoch 2/80: train_loss:0.08262385179599126
Epoch 3/80: train_loss:0.08269261568784714
Epoch 4/80: train_loss:0.08261758585770924
Epoch 5/80: train_loss:0.08258768916130066
Epoch 6/80: train_loss:0.08249961584806442
Epoch 7/80: train_loss:0.08240966002146403
Epoch 8/80: train_loss:0.0823507085442543
Epoch 9/80: train_loss:0.08223804086446762
Epoch 10/80: train_loss:0.08208281298478444
Epoch 11/80: train_loss:0.08203437676032384
Epoch 12/80: train_loss:0.08182398478190105
Epoch 13/80: train_loss:0.08171512186527252
Epoch 14/80: train_loss:0.08147646735111873
Epoch 15/80: train_loss:0.08118778963883717
Epoch 16/80: train_loss:0.08102376013994217
Epoch 17/80: train_loss:0.08073212206363678
Epoch 18/80: train_loss:0.08045539756615956
Epoch 19/80: train_loss:0.08018702765305837
Epoch 20/80: train_loss:0.079881156484286
Epoch 21/80: train_loss:0.07961425930261612
Epoch 22/80: train_loss:0.07918140292167664
Epoch 23/80: train_loss:0.07885564118623734


In [ ]:
method.ground.shape, method.anomalies.shape

((4499,), (2699,))

In [ ]:
### USAD
from AnomalyDetectionMethodClass import ADMethod_back
settings_={
    'EPOCHS':20,
    'BATCH_SIZE': 256,
    'SEQ_LEN': 12,
    'HIDDEN_SIZE': 100,
    'ALPHA': 0.5,
    'BETA': 0.5,
    'CONFIDENCE': 0.82,
    'DATASET': 'SWAT'}
data_path = ''
method_ = ADMethod_back('USAD',settings_,'')

method_.prepare_pipeline()
#h = method.train()
#scores, report = method.test()

/content/Time-Series-Anomaly-Detection-An-experimental-survey/USAD/USADSolver.py:22: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data()
/content/Time-Series-Anomaly-Detection-An-experimental-survey/USAD/USADSolver.py:22: DtypeWarning: Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data()


(494988, 12, 51)


In [ ]:
for [el] in method_.solver.train_loader:
  print(el.size())
  break
  

torch.Size([256, 612])


In [ ]:
for el in method.train_dl:
  print(el.size())
  break



torch.Size([256, 612])


In [ ]:
for el in method.solver.train_loader:
  print(el[0].shape)
  print(len(el))
  break

torch.Size([7919, 612])
1


In [ ]:
###  TRANSFORMER
from AnomalyDetectionMethodClass import ADMethod
data_path = './dataset/MSL'
settings = {
    'lr':1e-4,
    'num_epochs':3,
    'k':3,
    'win_size':100,
    'input_c':55,
    'output_c':55,
    'batch_size':256,
    'pretrained_model':None,
    'dataset':'MSL',
    'mode':'train', # choices=['train', 'test']
    'data_path':'./dataset/MSL',
    'model_save_path':'checkpoints',
    'anormly_ratio':1
}
method = ADMethod('transformer',settings, data_path)

method.prepare_pipeline()
method.train()
scores, report = method.test()

In [ ]:
### DEEPANT
from AnomalyDetectionMethodClass import ADMethod
# data_path = './NAB/realKnownCause/cpu_utilization_asg_misconfiguration.csv'
data_path = './dataset/MSL'


LR_LIST = [0.001, 0.0001, 0.00001, 0.000001]
SEQ_LIST = [100, 30, 100, 300]
for lr in [0.0001]:#LR_LIST:
  for seq in [100]: #SEQ_LIST:
    # wandb.init(project="experimental-survey-AD", entity="michiamoantonio", group='DeepAnt')
    # wandb.log({'lr': lr})
    # wandb.log({'seq_len': seq})
    settings = {
      'SEQ_LEN': seq,
      'out_dim': 1,
      'EPOCHS': 1,
      'LR': lr,
      'CONFIDENCE': 0.50,
      'TH_LIST': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
      'TH_SEARCH': True,
      'VERBOSE': True,
      'DATASET': 'SWAT'
    }
    # wandb.config = settings
    method = ADMethod('DeepAnt',settings, data_path)

    method.prepare_pipeline()
    l = method.train()
    scores, report = method.test()

    print(f"lr: {lr}, seq_len: {seq}")
    for i,el in enumerate(report): 
      print(f"#### th: {settings['TH_LIST'][i]}, f1-score: {el['macro avg']['f1-score']}")
      # wandb.log({'th': settings['TH_LIST'][i], 'f1-score': el['macro avg']['f1-score']})
    # wandb.finish()



LOAD TRAIN DATA


/content/Time-Series-Anomaly-Detection-An-experimental-survey/DeepAnt/DeepAntSolver.py:22: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  self.prepare_data(data_path)


SCALER
SCALED
window
WINDOWED


In [ ]:
!pip install torchinfo
from torchinfo import summary
summary(method.solver.model)